# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 27 2022 11:24AM,242847,10,0085911186,ISDIN Corporation,Released
1,Jul 27 2022 11:24AM,242847,10,0085911205,ISDIN Corporation,Released
2,Jul 27 2022 11:24AM,242847,10,0085911213,ISDIN Corporation,Released
3,Jul 27 2022 11:24AM,242847,10,0085911236,ISDIN Corporation,Released
4,Jul 27 2022 11:24AM,242847,10,0085911264,ISDIN Corporation,Released
5,Jul 27 2022 11:24AM,242847,10,0085911277,ISDIN Corporation,Released
6,Jul 27 2022 11:24AM,242847,10,0085911293,ISDIN Corporation,Released
7,Jul 27 2022 10:46AM,242845,10,MSP212590,"Methapharm, Inc.",Released
8,Jul 27 2022 10:46AM,242845,10,MSP212591,"Methapharm, Inc.",Released
9,Jul 27 2022 10:46AM,242845,10,MSP212580,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,242842,Released,4
29,242843,Executing,1
30,242844,Released,3
31,242845,Released,9
32,242847,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242842,NaN,NaN,4.0
242843,NaN,1.0,NaN
242844,NaN,NaN,3.0
242845,NaN,NaN,9.0
242847,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242781,3.0,3.0,8.0
242789,0.0,0.0,1.0
242791,0.0,0.0,1.0
242795,0.0,0.0,1.0
242798,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242781,3,3,8
242789,0,0,1
242791,0,0,1
242795,0,0,1
242798,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242781,3,3,8
1,242789,0,0,1
2,242791,0,0,1
3,242795,0,0,1
4,242798,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242781,3,3,8
1,242789,,,1
2,242791,,,1
3,242795,,,1
4,242798,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 27 2022 11:24AM,242847,10,ISDIN Corporation
7,Jul 27 2022 10:46AM,242845,10,"Methapharm, Inc."
16,Jul 27 2022 10:41AM,242844,10,"Methapharm, Inc."
19,Jul 27 2022 10:32AM,242843,20,"ACI Healthcare USA, Inc."
20,Jul 27 2022 10:29AM,242842,10,Emerginnova
24,Jul 27 2022 10:28AM,242841,19,"Graystone, LLC"
25,Jul 27 2022 10:02AM,242836,10,ISDIN Corporation
36,Jul 27 2022 10:02AM,242837,10,Beach Patient Assistance
37,Jul 27 2022 9:51AM,242835,15,"Carwin Pharmaceutical Associates, LLC"
42,Jul 27 2022 9:44AM,242808,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 27 2022 11:24AM,242847,10,ISDIN Corporation,,,7
1,Jul 27 2022 10:46AM,242845,10,"Methapharm, Inc.",,,9
2,Jul 27 2022 10:41AM,242844,10,"Methapharm, Inc.",,,3
3,Jul 27 2022 10:32AM,242843,20,"ACI Healthcare USA, Inc.",,1,
4,Jul 27 2022 10:29AM,242842,10,Emerginnova,,,4
5,Jul 27 2022 10:28AM,242841,19,"Graystone, LLC",,,1
6,Jul 27 2022 10:02AM,242836,10,ISDIN Corporation,,2,9
7,Jul 27 2022 10:02AM,242837,10,Beach Patient Assistance,,,1
8,Jul 27 2022 9:51AM,242835,15,"Carwin Pharmaceutical Associates, LLC",,,5
9,Jul 27 2022 9:44AM,242808,15,"Brookfield Pharmaceuticals, LLC",,,24


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 11:24AM,242847,10,ISDIN Corporation,7,,
1,Jul 27 2022 10:46AM,242845,10,"Methapharm, Inc.",9,,
2,Jul 27 2022 10:41AM,242844,10,"Methapharm, Inc.",3,,
3,Jul 27 2022 10:32AM,242843,20,"ACI Healthcare USA, Inc.",,1,
4,Jul 27 2022 10:29AM,242842,10,Emerginnova,4,,
5,Jul 27 2022 10:28AM,242841,19,"Graystone, LLC",1,,
6,Jul 27 2022 10:02AM,242836,10,ISDIN Corporation,9,2,
7,Jul 27 2022 10:02AM,242837,10,Beach Patient Assistance,1,,
8,Jul 27 2022 9:51AM,242835,15,"Carwin Pharmaceutical Associates, LLC",5,,
9,Jul 27 2022 9:44AM,242808,15,"Brookfield Pharmaceuticals, LLC",24,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 11:24AM,242847,10,ISDIN Corporation,7,,
1,Jul 27 2022 10:46AM,242845,10,"Methapharm, Inc.",9,,
2,Jul 27 2022 10:41AM,242844,10,"Methapharm, Inc.",3,,
3,Jul 27 2022 10:32AM,242843,20,"ACI Healthcare USA, Inc.",,1,
4,Jul 27 2022 10:29AM,242842,10,Emerginnova,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 11:24AM,242847,10,ISDIN Corporation,7.0,NaN,NaN
1,Jul 27 2022 10:46AM,242845,10,"Methapharm, Inc.",9.0,NaN,NaN
2,Jul 27 2022 10:41AM,242844,10,"Methapharm, Inc.",3.0,NaN,NaN
3,Jul 27 2022 10:32AM,242843,20,"ACI Healthcare USA, Inc.",NaN,1.0,NaN
4,Jul 27 2022 10:29AM,242842,10,Emerginnova,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 11:24AM,242847,10,ISDIN Corporation,7.0,0.0,0.0
1,Jul 27 2022 10:46AM,242845,10,"Methapharm, Inc.",9.0,0.0,0.0
2,Jul 27 2022 10:41AM,242844,10,"Methapharm, Inc.",3.0,0.0,0.0
3,Jul 27 2022 10:32AM,242843,20,"ACI Healthcare USA, Inc.",0.0,1.0,0.0
4,Jul 27 2022 10:29AM,242842,10,Emerginnova,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2428347,50.0,13.0,1.0
15,728450,31.0,17.0,0.0
19,1942469,19.0,10.0,0.0
20,728443,31.0,9.0,3.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2428347,50.0,13.0,1.0
1,15,728450,31.0,17.0,0.0
2,19,1942469,19.0,10.0,0.0
3,20,728443,31.0,9.0,3.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,50.0,13.0,1.0
1,15,31.0,17.0,0.0
2,19,19.0,10.0,0.0
3,20,31.0,9.0,3.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,50.0
1,15,Released,31.0
2,19,Released,19.0
3,20,Released,31.0
4,10,Executing,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20
Status,,,,
Completed,1.0,0.0,0.0,3.0
Executing,13.0,17.0,10.0,9.0
Released,50.0,31.0,19.0,31.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20
0,Completed,1.0,0.0,0.0,3.0
1,Executing,13.0,17.0,10.0,9.0
2,Released,50.0,31.0,19.0,31.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20
0,Completed,1.0,0.0,0.0,3.0
1,Executing,13.0,17.0,10.0,9.0
2,Released,50.0,31.0,19.0,31.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()